In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-186090
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-186090


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "SCComClPrjNew" #"GIVE_A_CLUSTER_NAME"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found the one I created")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target= ComputeTarget.create(ws,cluster_name,compute_config)
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())



Found the one I created
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-02-16T17:54:16.529000+00:00', 'errors': None, 'creationTime': '2022-02-16T17:44:37.821237+00:00', 'modifiedTime': '2022-02-16T17:44:41.390060+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
"--C" : choice(0.01,0.1,1) ,     
"--max_iter" : choice(10,20,40)
}
)  

# Specify a Policy
#BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=None, delay_evaluation=0)
policy =BanditPolicy(evaluation_interval=1, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#SKLearn(source_directory, *, compute_target=None, vm_size=None, vm_priority=None, entry_script=None, script_params=None, use_docker=True, custom_docker_image=None, image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None, _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False, _show_package_warnings=False)
#src = './training' ### YOUR CODE HERE ###
skl_estimator = SKLearn(source_directory="./",entry_script='train.py',compute_target=cluster_name)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#HyperDriveConfig(hyperparameter_sampling, primary_metric_name, primary_metric_goal, max_total_runs, max_concurrent_runs=None, max_duration_minutes=10080, policy=None, estimator=None, run_config=None, resume_from=None, resume_child_runs=None, pipeline=None, debug_flag=None)
#PrimaryMetricGoal(value) MAXIMIZE, MINIMIZE 
hyperdrive_config = HyperDriveConfig(
        hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs = 10, 
        max_concurrent_runs =4,
        policy=policy,
        estimator=skl_estimator

) ### YOUR CODE HERE ###


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_49aac923-1204-4b89-9ab0-88d0d434a6b7
Web View: https://ml.azure.com/runs/HD_49aac923-1204-4b89-9ab0-88d0d434a6b7?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-186090/workspaces/quick-starts-ws-186090&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-16T18:00:19.868464][API][INFO]Experiment created<END>\n""<START>[2022-02-16T18:00:20.793783][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-16T18:00:21.442864][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_49aac923-1204-4b89-9ab0-88d0d434a6b7
Web View: https://ml.azure.com/runs/HD_49aac923-1204-4b89-9ab0-88d0d434a6b7?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-186090/workspaces/quick-starts-ws-186090&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_49aac923-1204-4b89-9ab0-88d0d434a6b7',
 'target': 'SCComClPrjNew',
 'status': 'Completed',
 'startTimeUtc': '2022-02-16T18:00:19.589949Z',
 'endTimeUtc': '2022-02-16T18:07:00.723483Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd392fc50-8e8d-4657-83b9-63c84fe5f6d8',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '9',
  'score': '0.910773899848255',
  'best_child_run_id': 'HD_49aac923-1204-4b89-9ab0-88d0d434a6b7_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg186090.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_49aac923-1204-4b89-9ab0-88d0d434a6b7/azureml-l

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 

print("Best Run ID :", best_run.id)
print("\n Accuracy :", best_run_metrics['Accuracy'])

print("\n best run file names :",best_run.get_file_names())
print("\n best run details ", best_run.get_details())
print("\n best run metrics :",best_run_metrics)
######

Best Run ID : HD_49aac923-1204-4b89-9ab0-88d0d434a6b7_1

 Accuracy : 0.910773899848255

 best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']

 best run details  {'runId': 'HD_49aac923-1204-4b89-9ab0-88d0d434a6b7_1', 'target': 'SCComClPrjNew', 'status': 'Completed', 'startTimeUtc': '2022-02-16T18:02:24.366646Z', 'endTimeUtc': '2022-02-16T18:02:28.096762Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'd392fc50-8e8d-4657-83b9-63c84fe5f6d8', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': 

In [6]:
best_run_metrics

{'Regularization Strength:': 1.0,
 'Max iterations:': 40,
 'Accuracy': 0.910773899848255}

In [7]:
print("Best Run ID :", best_run.status)

Best Run ID : Completed


In [8]:
print("Best Run ID :", best_run.properties)

Best Run ID : {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'd392fc50-8e8d-4657-83b9-63c84fe5f6d8', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}


In [10]:
model = best_run.register_model(model_name='hyperdrive_best_run', 
model_path='outputs/modelSC.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy": best_run_metrics['Accuracy']})

#model = best_run.register_model(model_name='hyperdrive_best_run', 
#model_path='./modelSC.jolib', tags={"Method" : "HyperDrive"},
#properties={"Accuracy": best_run.get_metrics()["Accuracy"]})

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])


In [12]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


In [13]:
concat_x_y = pd.concat([x,y], axis=1)
TRAIN_d, TEST_d = train_test_split(concat_x_y, test_size=0.1, random_state=24)
TRAIN_d.to_csv('training/TRAIN_CSV.csv')


In [14]:

ds_new = ws.get_default_datastore()


In [15]:
ds_new.upload(src_dir='training',target_path='TRAIN_DATA')



Uploading an estimated of 1 files
Uploading training/TRAIN_CSV.csv
Uploaded training/TRAIN_CSV.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_2e18d19f371e4debb923d65069ba0d9f

In [16]:
TRAIN_DATASET = Dataset.Tabular.from_delimited_files(path=[(ds_new, ('TRAIN_DATA/TRAIN_CSV.csv'))])

In [27]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=TRAIN_DATASET,
    label_column_name='y',
    n_cross_validations=4,
    ebable_early_stopping=True,
    enable_onnx_compatible_models=True
    )

In [28]:
# Submit your automl run
#helpp from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

### YOUR CODE HERE ###
remote_run = exp.submit(config= automl_config, show_output= True)



No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6d6e0318-e26d-48d2-9f70-ee29fd145d87,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [29]:
# Retrieve and save your best automl model.
# help from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

### YOUR CODE HERE ###
automl_best_run, automl_fitted_model = remote_run.get_output()

print(hasattr(automl_fitted_model, 'steps'))
print("\n")
print(automl_best_run.get_metrics())
print("\n")
print(automl_best_run.get_tags())

True


{'balanced_accuracy': 0.7210822465377305, 'f1_score_micro': 0.9157983166733934, 'log_loss': 0.21837044731093552, 'norm_macro_recall': 0.4421644930754608, 'average_precision_score_micro': 0.9816855420615271, 'weighted_accuracy': 0.9641998612699353, 'precision_score_weighted': 0.9070937721556721, 'precision_score_macro': 0.8083158706130548, 'recall_score_micro': 0.9157983166733934, 'precision_score_micro': 0.9157983166733934, 'f1_score_weighted': 0.9088589950818158, 'accuracy': 0.9157983166733934, 'AUC_micro': 0.9808623891550036, 'matthews_correlation': 0.5218062954016959, 'AUC_weighted': 0.9485655553070313, 'AUC_macro': 0.9485655553070315, 'recall_score_weighted': 0.9157983166733934, 'average_precision_score_weighted': 0.9562381448611927, 'recall_score_macro': 0.7210822465377305, 'f1_score_macro': 0.7544235894006752, 'average_precision_score_macro': 0.827774114821568, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_6d6e0318-e26d-48d2-9f70-ee29fd145d87_12/confusion

In [30]:
# Save your best automl model.
#save_onnx_model(onnx_model: onnx.onnx_ml_pb2.ModelProto, file_path: str) -> None 
#help from https://docs.microsoft.com/en-us/python/api/azureml-automl-runtime/azureml.automl.runtime.onnx_convert.onnx_converter.onnxconverter?view=azure-ml-py
#help from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings


from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run, automl_fitted_model = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model, './outputs/Automl_Model.onnx')

In [31]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

